In [ ]:
#| default_exp arena_agent
#| test: false


#"Chat UI + LLM agent that talks to FastMCP tools."


In [ ]:
#| export
#| eval: false

from __future__ import annotations
import asyncio, html, json, os
from typing import AsyncIterator, List, Dict, Tuple, Any

from fastapi import FastAPI, Request, status
from fastapi.responses import HTMLResponse
from fasthtml import FastHTML
from fasthtml.common import Div, Form, Input, Button, H1
from sse_starlette.sse import EventSourceResponse
from agents import Agent, Runner
from agents.mcp import MCPServerSse
from fasthtml.common import (Body, Button, Div, Form, Group, H1, H2, Input,
                             Link, NotStr, Script, Style)
from typing import List
import datetime
from dataclasses import dataclass
import httpx, asyncio, yaml, re
from functools import lru_cache
from typing import List, Dict
from agents import ToolCallItem
from monsterui.all import Theme 


In [ ]:
#| export
#| eval: false

# ── Config (env-vars for docker-compose) ───────────────────────────────────
MCP_URL        = os.getenv("MCP_URL", "http://tools:9001/sse")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")


In [ ]:
#| export
#| eval: false

history: list[dict] = []

In [ ]:
#| export
#| eval: false
 
@dataclass
class EventContext:
    event_date_time: datetime
    event_title: str
    event_description: str
    
EventContext = EventContext(event_date_time="04.10.2025 12:00:00", 
                            event_title="Hockey match KIEKKO-ESPOO vs KÄRPÄT", 
                            event_description=
                            "Liiga regular-season showdown at Metro Areena • Doors open 11:15 "                        "Sustainable transport encouraged "                                                        "(metro: Urheilupuisto, bike racks outside Gate B).")

In [ ]:
#| export
#| eval: false

# ── FastHTML shell with Tailwind + HTMX + SSE ext ─────────────────────────
app_html = FastHTML(
    hdrs=Theme.zinc.headers() + [
        Script(src="https://cdn.tailwindcss.com"),

        # daisyUI (optional)
        Script(src="https://cdn.jsdelivr.net/npm/daisyui@4.10.2/dist/full.min.js"),

        # HTMX SSE extension (HTMX core auto-injected via live=True)
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.3/dist/sse.js"),

    ],  
    live=True,
    html_attrs={"data-theme": "dark", "class": "bg-gray-50 text-gray-700"},
)

# FastAPI wrapper so uvicorn can find the ASGI app
app = FastAPI(title="Arena Buddy", docs_url=None)
app.mount("/", app_html)


In [ ]:
#| export
#| eval: false

# ── In-memory chat log ────────────────────────────────────────────────────
MSG: List[Dict[str, str]] = []
MSG_LOCK = asyncio.Lock()


In [ ]:
#| export
#| eval: false

# ── UI helpers ─────────────────────────────────────────────────────────────
def _chat_bubble(idx: int, **hx):
    role, txt = MSG[idx]["role"], MSG[idx]["content"] or "…"
    side   = "chat-end" if role == "user" else "chat-start"
    bubble = "bg-sky-700 text-white" if role == "assistant" else "bg-gray-200"
    return Div(
        Div(role, cls="chat-header text-xs text-gray-500"),
        Div(txt if role == "user" else html.unescape(txt),
            cls=f"chat-bubble {bubble}", **hx),
        cls=f"chat {side}", id=f"m{idx}",
    )

def _chat_input():
    return Input(
        id="msgin",                    
        name="msg",
        type="text",
        autocomplete="off",
        placeholder="Type your question…",
        cls="input input-bordered w-full",
        hx_swap_oob="true",             
        onkeyup="event.key==='Enter' && this.form.requestSubmit()",
    )



In [ ]:
#| export
#| eval: false

# ── Home page ─────────────────────────────────────────────────────────────
@app_html.get("/")
async def home():
    ui = Div(
        Div(id="toaster", cls="toast toast-top toast-end fixed z-50"),
        H1("Arena Buddy", cls="text-3xl font-bold mb-4"),
        Div(id="chatlog",
            cls="space-y-3 mb-4 h-[70vh] overflow-y-auto bg-base-200 p-4 rounded-box"),
        Form(
            Div(_chat_input(),
                Button("Send ✈", cls="btn btn-primary ml-2"),
                cls="flex"),
            hx_post="/send",          
            hx_target="#chatlog",
            hx_swap="beforeend",
        ),
        cls="max-w-2xl mx-auto p-6",
    )
    return ui


In [ ]:
#| export
#| eval: false
mainAgent_instruction = f"""
╭─────────────────────────────────────────────────────────────╮
│ 🎟  EVENT CONTEXT                                           │
╰─────────────────────────────────────────────────────────────╯
• **When**   : {EventContext.event_date_time}
• **What**   : {EventContext.event_title}
• **Details**: {EventContext.event_description}

Your single goal → get the user there **on time** with the **lowest possible CO₂** footprint.

────────────────────────────────────────────────────────────────
HOW TO THINK & RESPOND
────────────────────────────────────────────────────────────────
1. **Understand** the latest user message + full history.
2. **Plan** a short chain-of-thought *silently* (don’t reveal it).
3. **Pick or mount tools**
   • If an existing tool fits (e.g. `walk_route`, `bike_route`,
     `pt_route`, `car_route`, `nearest_parking_ids`, etc.) – call it.
   • Else call  
     `quick_mount_openapi("<keywords>")`  
     then immediately call the needed REST operation.
4. **Compare modes**  
   • If any proposed leg is high-carbon (car, taxi, flight),
     suggest at least one greener alternative and **quantify the saving**
     (e.g. “Train cuts ≈80 % CO₂ vs car for this distance”).
5. **Summarise results** in clear bullet points:
   • departure / arrival time, duration, CO₂ estimate, cost (if known).
6. **Humour & tone**  
   • Every 3-4 replies, add a light joke or emoji (PG-13, relevant).
7. **Car users**  
   • If the user insists on driving, call
     `nearest_parking_ids` ➜ `parking_status`
     and gently remind why public transport is greener.
8. **Error handling**  
   • If a tool raises *ToolError* mentioning “API key”,
     politely ask the user for the key **or** suggest a free workaround.
9. **Final formatting**
   • After you have the textual answer, wrap it in a MonsterUI block:
     ```html
     <div class="card shadow-lg bg-base-200">
       … route summary …
       <button class="btn btn-primary">Open map</button>
     </div>
     ```
   • Use simple MonsterUI / DaisyUI classes (`card`, `btn`, `badge`, …).
10. **Be concise** – maximum 4-6 sentences + the card.

(You may omit steps that are not relevant to the user’s request.)
"""



In [ ]:
#| export
#| eval: false


from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, Usage, function_tool
from typing import Callable, Any

class ToolChatHook(RunHooks[None]):
    """
    Fires push(msg) on every tool start / end / error so the UI can
    display a live notification in the chat area.
    """
    def __init__(self, push: Callable[[str], None]):
        self._push = push
        
    async def on_start(self, context, agent) -> None:
        """Called once when the whole agent run begins."""
        self._push("🤖 _thinking…_")

    async def on_end(self, context, agent, result) -> None:
        """Called once after the final answer has been produced."""
        # Nothing fancy for now; you could log token usage here
        pass        
        

    async def on_tool_start(self, context, agent, tool) -> None:
       self._push(f"{tool.name}  started")

    async def on_tool_end(self, context, agent, tool, result) -> None:
        self._push(f"✔ {tool.name} finished")
        self._push(f"result: {result}")
        

    async def on_tool_error(self, context, agent, tool, error) -> None:
        self._push(f"⚠ {tool.name} failed: {error}")

In [ ]:
#| export
#| eval: false


from collections import deque
from agents import Agent, Runner, trace
from agents.mcp import MCPServerSse

async def _assistant_html(user_prompt: str, push: Callable[[str], None]) -> tuple[str, list[str]]:
    """
    ①   mount / call tools as needed
    ②   craft the plain-text answer
    ③   wrap it in MonsterUI HTML
    returns (html_block, ["tool: output", …])
    """
  

    async with MCPServerSse(name="ui", params={"url": MCP_URL}, client_session_timeout_seconds=30) as srv:
        agent = Agent(
            name="Main Agent",
            instructions=mainAgent_instruction,
            mcp_servers=[srv],
            hooks=ToolChatHook(push),
            model="o3"
        )

        # ── build the initial input list (first user turn) ───────────────
        input_items: list = [{"role": "user", "content": user_prompt}]

        with trace(workflow_name="Arena-3-step"):
            # ①  Mount or use step
            input_items.append({
                "role": "system",
                "content": (
                    "If no suitable tool exists, call quick_mount_openapi, "
                    "then the required operation. Summarise the result."
                ),
            })
            res1 = await Runner.run(agent, input_items)


            # ②  Natural-language answer
            input_items = res1.to_input_list()         
            res2 = await Runner.run(agent, input_items)


            # ③  MonsterUI formatting
            input_items = res2.to_input_list()
            input_items.append({
                "role": "system",
                "content": (
                    "Take the previous assistant answer and wrap it in a MonsterUI card.\n"
        "—  use <div class='card bg-base-300 text-base-100 shadow-lg p-4 space-y-3'>\n"
        "—  put the answer itself inside <p class='whitespace-pre-wrap'> … </p>\n"
        "—  try to make every time different cards \n"
                    "- add funny images connected to hokey, CO2 etc \n"
        "—  if you need an action, add\n"
        "     <button class='btn btn-primary' "
        "             hx-post='/open-map' hx-target='#toaster' hx-swap='afterbegin'>"
        "Open map 🗺️</button>\n"
        "Return **HTML only** – no markdown fences, no extra text."
                ),
            })
            res3 = await Runner.run(agent, input_items)
            

    return res3.final_output


In [ ]:
#| export
#| eval: false

# ── /send endpoint ────────────────────────────────────────────────────────
@app_html.post("/send")
async def send(request: Request):
    form   = await request.form()
    prompt = str(form.get("msg", "")).strip()
    if not prompt:
        return HTMLResponse("", status_code=status.HTTP_204_NO_CONTENT)

    async with MSG_LOCK:
        MSG.extend([
            {"role": "user",      "content": html.escape(prompt)},
            {"role": "assistant", "content": ""},
        ])
        idx_user, idx_asst = len(MSG) - 2, len(MSG) - 1

    return (
        _chat_bubble(idx_user).__html__() +
        _chat_bubble(
            idx_asst,
            hx_ext="sse",
            sse_connect=f"/stream/{idx_asst}",
            sse_swap="message",
            sse_close="close",
            hx_swap="innerHTML",
        ).__html__() +
        _chat_input().__html__()
    )


In [ ]:
#| export
#| eval: false

from starlette.responses import StreamingResponse     

# ── helpers --------------------------------------------------------
def _sse(event: str, payload: str) -> str:
    """
    Return one correctly-formatted Server-Sent-Events block.

    Each logical message must be terminated with a *blank* line, otherwise
    the browser keeps buffering and the event never reaches the JS side.
    """
    # HTMX’ sse.js is happy with plain HTML, so we don’t wrap in JSON here.
    body = "\n".join(f"data: {line}" for line in payload.splitlines())
    return f"event: {event}\n{body}\n\n"


async def _stream_reply(idx: int) -> AsyncIterator[str]:  # noqa: C901 – acceptable
    async with MSG_LOCK:
        if idx <= 0 or idx >= len(MSG):
            return 
    prompt_html = MSG[idx - 1]["content"]

    q: asyncio.Queue[str] = asyncio.Queue()

    # push() will be handed to ToolChatHook and to the agent itself
    def push(msg: str) -> None:  
        q.put_nowait(_sse("message", msg))

    async def _run() -> None:
        try:
            reply_html = await _assistant_html(prompt_html, push)
        except Exception as exc:
            reply_html = f"⚠ Internal error: {exc}"
        await q.put(_sse("message", reply_html))     # final HTML card
        await q.put("event: close\ndata:\n\n")       # tell HTMX to close

    asyncio.create_task(_run()) 
    
    # ⬅️  *this* is what StreamingResponse must consume
    async def streamer() -> AsyncIterator[str]:
        while True:                   # blocks until the queue gets data
            yield await q.get()

    return streamer()                 # ← DON’T forget this!

# ── /stream/{idx} endpoint (SSE) ------------------------------------------
@app_html.get("/stream/{idx}")
async def stream(idx: int):
    """
    Streaming endpoint used by the chat bubbles (`hx-ext="sse"`).
    HTMX opens the connection, waits for the first “message” event,
    swaps the payload into the bubble, then receives a “close” event
    and disposes the EventSource.
    """
    generator = await _stream_reply(idx)   # get the AsyncIterator ✅
    if generator is None:                  # invalid idx guard
        return HTMLResponse(
            status_code=status.HTTP_204_NO_CONTENT
        )
    return StreamingResponse(
       generator,
        media_type="text/event-stream",
    )


In [ ]:
#| export
#| eval: false

from monsterui.franken import ModalHeader, ModalBody, Modal
from fastapi.responses import HTMLResponse

import json

LEAFLET_CSS  = "https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"
LEAFLET_JS   = "https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"

@app_html.post("/open-map")
async def open_map(latlngs: list[list[float]] = Form(...)) -> HTMLResponse:
    """
    `latlngs` must be a list like [[lat, lon], [lat, lon], …]
    (HTMX will POST it as a JSON string – see §2 below).
    """
    coords_js = json.dumps(latlngs)     # embed safely in <script>

    html = Modal(
        ModalHeader("Route preview 🗺️"),
        ModalBody(
            "<div id='map' class='w-full h-80 rounded-xl'></div>",
            as_html=True
        ),
        open=True,                          # open immediately
    ).__html__() + f"""
    <link rel="stylesheet" href="{LEAFLET_CSS}"/>
    <script src="{LEAFLET_JS}"></script>
    <script>
      (function () {{
        const coords = {coords_js};
        const map = L.map('map', {{ zoomControl:false }})
                     .fitBounds(coords);
        L.tileLayer('https://tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png',
                     {{ attribution:'© OSM' }}).addTo(map);
        L.polyline(coords, {{ color:'#2563eb', weight:5 }}).addTo(map);
      }})();
    </script>
    """

    return HTMLResponse(html, status_code=200, media_type="text/html")
